In [1]:
import os
import glob
from tqdm import tqdm

import pandas as pd

In [6]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/"

In [17]:
df_l = []

for file in tqdm(glob.glob(os.path.join(root, "*", "metadata.csv"))):
    dataset_folder = file.split("/")[-2]
    df = pd.read_csv(file)
    print(f"{dataset_folder}: {len(df)}")

    if dataset_folder == "rsna-str-pulmonary-embolism-detection":
        df["path"] = df["path"].apply(
            lambda x: x.replace("/train/", "/train-png/"))
        df["path"] = df["path"].apply(
            lambda x: x.replace("/test/", "/test-png/"))
        df["path"] = df["path"].apply(lambda x: x.replace(".dcm", ".png"))
    # if dataset_folder == "siim-covid19-detection":
    #     df["class"] = df["class"].apply(lambda x: x).replace("typical appearance pneumonia", "pneumonia")
    #     df["class"] = df["class"].apply(lambda x: x).replace("indeterminate appearance pneumonia", "pneumonia")
    #     df["class"] = df["class"].apply(lambda x: x).replace("atypical appearance pneumonia", "pneumonia")
    #     df["split"] = "test"

    df["path"] = df["path"].apply(lambda x: os.path.join(
        dataset_folder, x.replace("./", "")))

    df_l.append(df)

df_combined = pd.concat(df_l, ignore_index=True, sort=False)

  0%|          | 0/7 [00:00<?, ?it/s]

rsna-str-pulmonary-embolism-detection: 1790594


 14%|█▍        | 1/7 [00:08<00:48,  8.05s/it]

siim-covid19-detection: 6334


 43%|████▎     | 3/7 [00:08<00:09,  2.38s/it]

...

CC-CCII: 411529


 71%|███████▏  | 5/7 [00:11<00:03,  1.81s/it]

Lung-PET-CT-Dx: 204836


100%|██████████| 7/7 [00:12<00:00,  1.82s/it]


vinbigdata-chest-xray-abnormalities-detection: 15000


In [8]:
df_combined["path"]

0          rsna-str-pulmonary-embolism-detection/train-pn...
1          rsna-str-pulmonary-embolism-detection/train-pn...
2          rsna-str-pulmonary-embolism-detection/train-pn...
3          rsna-str-pulmonary-embolism-detection/train-pn...
4          rsna-str-pulmonary-embolism-detection/train-pn...
                                 ...                        
2937340    vinbigdata-chest-xray-abnormalities-detection/...
2937341    vinbigdata-chest-xray-abnormalities-detection/...
2937342    vinbigdata-chest-xray-abnormalities-detection/...
2937343    vinbigdata-chest-xray-abnormalities-detection/...
2937344    vinbigdata-chest-xray-abnormalities-detection/...
Name: path, Length: 2937345, dtype: object

Check the existance of every image file

In [10]:
df_combined.to_csv(os.path.join(root, "metadata.csv"), index=False)

In [11]:
dict(df_combined["dataset"].value_counts())

{'rsna-pulmonary-embolism': 1790594,
 'CC-CCII': 411529,
 'mimic-cxr': 320912,
 'lung-pet-ct-dx': 204836,
 'chexpert': 188140,
 'vinbigdata-cxr': 15000,
 'ssim-covid19': 6334}

In [18]:
classes = dict()

for cls in df_combined["class"].tolist():
    for x in cls.split(","):
        x = x.strip()
        if x not in classes.keys():
            classes[x] = 1
        else:
            classes[x] += 1

classes

{'no findings': 2399485,
 'pulmonary embolism': 96540,
 'typical appearance pneumonia': 39901,
 'indeterminate appearance pneumonia': 1108,
 'atypical appearance pneumonia': 22355,
 'cardiomegaly': 93646,
 'pleural effusion': 164176,
 'atelectasis': 147884,
 'edema': 121203,
 'consolidation': 29811,
 'pneumothorax': 33801,
 'adenocarcinoma': 19927,
 'squamous cell carcinoma': 7081,
 'small cell carcinoma': 3034,
 'large cell carcinoma': 201,
 'pleural thickening': 1981,
 'aortic enlargement': 3067,
 'pulmonary fibrosis': 1617,
 'ild': 386,
 'nodule/mass': 826,
 'other lesion': 1134,
 'lung opacity': 1322,
 'infiltration': 613,
 'calcification': 452}

In [5]:
df = pd.read_csv(os.path.join(root, "metadata_v1.csv"))

In [7]:
df_combined.drop_duplicates(subset=["id", "class"])

,id,exam_id,path,class,exam_class,dataset,class_num,contains_negative,modality,split
0,8cac6d50ca40/3201d1c46942/0640afc508cf,8cac6d50ca40/3201d1c46942,rsna-str-pulmonary-embolism-detection/train-pn...,no findings,no findings,rsna-pulmonary-embolism,2,1,ct,train
1,eb026d364980/ae8d98f4080a/28039e2a8eb8,eb026d364980/ae8d98f4080a,rsna-str-pulmonary-embolism-detection/train-pn...,no findings,no findings,rsna-pulmonary-embolism,2,1,ct,train
2,848ecdaf1c61/aeebe4cac843/af90dc5c53eb,848ecdaf1c61/aeebe4cac843,rsna-str-pulmonary-embolism-detection/train-pn...,no findings,no findings,rsna-pulmonary-embolism,2,1,ct,train
3,1a59a1ce6905/100a8d863273/73ef1600bbea,1a59a1ce6905/100a8d863273,rsna-str-pulmonary-embolism-detection/train-pn...,no findings,no findings,rsna-pulmonary-embolism,2,1,ct,train
4,86bbf267dba4/dd6208d6960e/0095dd4dd748,86bbf267dba4/dd6208d6960e,rsna-str-pulmonary-embolism-detection/train-pn...,no findings,no findings,rsna-pulmonary-embolism,2,1,ct,train
...,...,...,...,...,...,...,...,...,...,...
3186683,1.3.6.1.4.1.14519.5.2.1.6655.2359.433326537115...,1.3.6.1.4.1.14519.5.2.1.6655.2359.220574572626...,Lung-PET-CT-Dx/manifest/Lung-PET-CT-Dx/Lung_Dx...,no findings,no findings,lung-pet-ct-dx,4,1,ct,test
3186684,1.3.6.1.4.1.14519.5.2.1.6655.2359.143205918693...,1.3.6.1.4.1.14519.5.2.1.6655.2359.261661225897...,Lung-PET-CT-Dx/manifest/Lung-PET-CT-Dx/Lung_Dx...,no findings,no findings,lung-pet-ct-dx,4,1,ct,train
3186685,1.3.6.1.4.1.14519.5.2.1.6655.2359.138418961766...,1.3.6.1.4.1.14519.5.2.1.6655.2359.271888981993...,Lung-PET-CT-Dx/manifest/Lung-PET-CT-Dx/Lung_Dx...,no findings,no findings,lung-pet-ct-dx,4,1,ct,train
3186686,1.3.6.1.4.1.14519.5.2.1.6655.2359.169684324288...,1.3.6.1.4.1.14519.5.2.1.6655.2359.506152635744...,Lung-PET-CT-Dx/manifest/Lung-PET-CT-Dx/Lung_Dx...,no findings,no findings,lung-pet-ct-dx,4,1,ct,train
